In [ ]:
# 01 - Params

url_base = "https://www.twingalaxies.com/showthread.php/176004-Dispute-Jeremy-Young-Arcade-Donkey-Kong-Points-Hammer-Allowed-Player-Billy-L-Mitchell-Score-1-062-800/page"
file_name = "./twingalaxies_dispute_billymitchell.xls"
file_name_linklist = "./twingalaxies_dispute_billymitchell_linklist.csv"

# 02 - Definitions and imports

import urllib
import pandas as pd
import bs4
import time
import re
import dateutil
import collections
import datetime

def get_post_id(post_parent):
        #print(post_parent)
        try:
            post_counter = post_parent.find("a", {"class": "postcounter"})
            return post_counter.contents[0]
        except AttributeError:
            return None

        
def get_thanks_or_likes(soup, str_thanks_or_likes):
    out_dict = collections.defaultdict(list)
    contents = soup.findAll('img', {"alt": str_thanks_or_likes})
    for rec_content in contents:
        if "others</a>" in str(rec_content.parent):
            k = rec_content.parent.find("a", {"href": re.compile("^thanks.php?")})
            url_thanks = "https://www.twingalaxies.com/" + k.attrs["href"]

            page_thanks = urllib.request.urlopen(url_thanks)
            soup_thanks = bs4.BeautifulSoup(page_thanks, 'html.parser')

            try:
                hh = [x.contents[0] for x in soup_thanks.findAll("a", {"href": re.compile("^member.php")})]
            except IndexError:
                hh = [x.contents[0] for x in rec_content.parent.findAll("a", {"href": re.compile("^member.php?")})]
        else:
            hh = [x.contents[0] for x in rec_content.parent.findAll("a", {"href": re.compile("^member.php?")})]

        out_list = [x if type(x) is bs4.element.NavigableString else x.contents[0] for x in hh]

        parent_str = ""
        post_id = None
        while post_id is None:
            parent_str += ".parent"
            exec_str = "post_id = get_post_id(rec_content" + parent_str + ")"
            ldict = locals()
            exec(exec_str)
            post_id = ldict["post_id"]
        
        out_dict[post_id] = out_list
        
    return out_dict


def read_thread_df(file_name):
    try:
        all_loaded_df = pd.read_excel(file_name)
    except FileNotFoundError:
        return None
        
    for col_name in all_loaded_df.columns:
        if "_list" in col_name:
            all_loaded_df.loc[:, col_name] = all_loaded_df.loc[:, col_name].apply(lambda x: x.split(", ") if not pd.isnull(x) else [])

    return all_loaded_df


def write_thread_df(all_df, file_name):

    all_listasstrings_df = all_df.copy()

    for col_name in all_listasstrings_df.columns:
        if "_list" in col_name:
            all_listasstrings_df.loc[:, col_name] = all_listasstrings_df.loc[:, col_name].apply(lambda x: re.sub("\[|\]|\'", "", str(x)))

    all_listasstrings_df.to_excel(file_name, index=None)
    all_listasstrings_df.to_csv(file_name.replace("xls", "csv"), sep=";", quoting=2, encoding="utf-8", index=None)


def update_thread_df(all_df, url_base):
    
    def parse_postid_to_int(post_id):  
        return int(post_id.replace("#", ""))
    
    if all_df is None:
        col_list = ["post_id", "post_time", "url_post", "user_name", "post_score", "post_rank", "content", "links_list", "links_number", "cites_list", "cites_number", "thanks_list", "thanks_number", "likes_list", "likes_number"]
        all_df = pd.DataFrame(columns=col_list)
        post_to_parse = 1
        page_to_parse = 1
    else:
        post_last_read = all_df.tail(1)["post_id"].iloc[0]
        post_to_parse = parse_postid_to_int(post_last_read) + 1
        page_to_parse = int(post_to_parse / 10) + 1

    for page_number in range(page_to_parse, 1000, 1):

        url_rec = url_base + str(page_number)
        page = urllib.request.urlopen(url_rec)
        soup = bs4.BeautifulSoup(page, 'html.parser')

        page_last_number = int(soup.findAll("a", {"class": "popupctrl jmppop"})[0].contents[0].split(" ")[-1])
        if page_number > page_last_number:
            break

        print("Parsing page " + str(page_number))
        time.sleep(1)

        contents = soup.findAll('div', id=re.compile('^post_message'))
        users = soup.findAll('div', {"class": re.compile('msg-user comments')})
        assert len(contents) == len(users)
        rec_dict = {}

        likes_list = get_thanks_or_likes(soup, "Likes")
        thanks_list = get_thanks_or_likes(soup, "Thanks")

        for post_id_in_page in range(len(contents)):

            post_parent = contents[post_id_in_page].parent.parent.parent.parent.parent
            post_counter = post_parent.find("a", {"class": "postcounter"})
            rec_dict["post_id"] = get_post_id(post_parent)
            if post_to_parse > parse_postid_to_int(rec_dict["post_id"]):
                continue

            rec_dict["url_post"] = "https://www.twingalaxies.com/" + post_counter.attrs["href"]

            d = [x for x in contents[post_id_in_page].contents if x != "\n"][0]
            e = [x for x in d.contents if x != "\n"]
            content_formatted = "\n".join([str(x).replace("\t", "") for x in e])
            content_formatted = re.sub("(^\r\n)|(\r\n$)|(\r)", "", content_formatted)
            content_formatted = content_formatted.replace('<div class="bbcode_quote">', "\n##### START OF QUOTE #####\n").replace("</div>\n</div>\n</div>\n</div>\n", "\n##### END OF QUOTE #####\n")

            content_soup = bs4.BeautifulSoup(content_formatted, "lxml")
            for match in content_soup.findAll('span'):
                match.unwrap()

            content_formatted_nohtml = " ".join(content_soup.strings)

            rec_dict["content"] = re.sub("\n+", " \n ", re.sub("^\n|\n$", "", re.sub("[ ]*\n+[ ]*", "\n", content_formatted_nohtml))) if isinstance(content_formatted_nohtml, str) else content_formatted_nohtml

            content_no_double_spaces = rec_dict["content"].replace("  ", " ")
            citations_in_str = [m.end() for m in re.finditer('Originally Posted by ', content_no_double_spaces)]
            rec_dict["cites_list"] = [content_no_double_spaces[idx:].split(" ")[0] for idx in citations_in_str]
            rec_dict["cites_number"] = len(rec_dict["cites_list"])

            post_time_list = post_parent.find("span", {"class": "date"}).contents

            date_today = datetime.datetime.now()
            date_yesterday = date_today - datetime.timedelta(days=1)
            time_str = post_time_list[0].replace(",\xa0", "") + " " +  post_time_list[1].contents[0]
            time_str = time_str.replace("Yesterday", date_yesterday.strftime("%Y-%m-%d")).replace("Today", date_today.strftime("%Y-%m-%d"))
            rec_dict["post_time"] = dateutil.parser.parse(time_str)

            user = [x for x in users[post_id_in_page].contents if x != "\n"][0]
            str_start = ".val('"
            idx_start = str(user).find(str_start) + len(str_start)
            idx_end = idx_start + str(user)[idx_start:].find("'")
            rec_dict["user_name"] = str(user)[idx_start:idx_end]

            rec_dict["likes_list"] = [x for x in likes_list[rec_dict["post_id"]] if x != rec_dict["user_name"]]
            rec_dict["thanks_list"] = [x for x in thanks_list[rec_dict["post_id"]] if x != rec_dict["user_name"]]

            rec_dict["likes_number"] = len(rec_dict["likes_list"])
            rec_dict["thanks_number"] = len(rec_dict["thanks_list"])

            idx_start_list = [m.start() for m in re.finditer('href="htt', content_formatted)]
            link_list = [content_formatted[idx:].split('"')[1] for idx in idx_start_list]

            idx_start_list = [m.start() for m in re.finditer('src="htt', content_formatted)]
            link_list += [content_formatted[idx:].split('"')[1] for idx in idx_start_list]

            link_list_sel = [link for link in link_list if "https://www.twingalaxies.com/showthread.php?p" not in link]
            rec_dict["links_list"] = list(set(link_list_sel))

            rec_dict["links_number"] = len(rec_dict["links_list"])

            all_df = all_df.append(rec_dict, ignore_index=True)
            
        write_thread_df(all_df, file_name)
            
    return all_df

# 03 - Programme

all_df = read_thread_df(file_name)
all_df = update_thread_df(all_df, url_base)
write_thread_df(all_df, file_name)

all_df.tail(3)